In [ ]:
import os, json
from dotenv import load_dotenv

load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
import openai
client = openai.OpenAI()

PROMPT = """
I have the following function

`get_weather`
`get_currecy`
`get_news`

All of them receive the name of a country as an arguemtn (i.e get_news('Spain))

Please anser with the name of the function that you would like me to run.

Please say nothing else, just the name of the functino with the arguemtns.

Answer th following question: What is the capital of Korea?

"""
response = client.chat.completions.create(
	model="gpt-4o-mini",
	messages=[{
		"role" : "user",
		"content" : PROMPT
	}]

)

response

In [ ]:
message =response.choices[0].message.content

message

In [1]:
import openai, json

client = openai.OpenAI()
messages = []

In [2]:
def get_weather(city):
	return "33 degrees celcius."


FUNCTION_MAP = {
	"get_weather": get_weather,
}

In [3]:
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "A function to get the weather of a city.",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "The name of the city to get the weather of.",
                    }
                },
                "required": ["city"],
            },
        },
    }
]

In [4]:
from openai.types.chat import ChatCompletionMessage

def process_ai_response(message: ChatCompletionMessage):
	if message.tool_calls:
		messages.append({
			"role" : "assistant",
			"content" : message.content or "",
			"tool_calls" : [
				{
					"id" : tool_call.id,
					"type" : "function",
					"function" : {
						"name" : tool_call.function.name,
							"arguments" : tool_call.function.arguments
						}
					} for tool_call in message.tool_calls
				],
			}
		)

		for tool_call in message.tool_calls:
			function_name = tool_call.function.name
			arguments = tool_call.function.arguments

			print(f"Calling function: {function_name} with arguments {arguments}")

			try :
				arguments = json.loads(arguments)
			except json.JSONDecodeError:
				arguments = {}

			function_to_run = FUNCTION_MAP.get(function_name)

			result = function_to_run(**arguments)

			print(f"Ran {function_name} with args {arguments} for a result of {result}")

			messages.append({
				"role" : "tool",
				"tool_call_id" : tool_call.id,
				"name" : function_name,
				"content" : result
			})
		call_ai()
		
	else :
		messages.append({"role" : "assistant", "content" : message.content})
		print(f"AI: {message.content}")

def call_ai():
		response = client.chat.completions.create(
			model="gpt-4o-mini",
			messages=messages,
			tools=TOOLS,
		)
		process_ai_response(response.choices[0].message)

In [5]:
while True:
	message = input("Send a message to LLM...")
	if message == 'quit':
		break
	else:
		messages.append({
			"role" : "user",
			"content" : message
		})
		print(f"User: {message}")
		call_ai()

User: what is the weather in Korea?
Calling function: get_weather with arguments {"city":"Seoul"}
Ran get_weather with args {'city': 'Seoul'} for a result of 33 degrees celcius.
AI: The weather in Seoul, Korea is 33 degrees Celsius.
User: wow Awesome
AI: I'm glad you think so! If you have any more questions about the weather or anything else, feel free to ask!


In [ ]:
messages